In [ ]:
import json
import os
from datetime import datetime
from http import HTTPStatus

import numpy as np
import pandas as pd
import requests
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo import evo_common
from evo.blockmodels import BlockModelAPIClient
from evo.common.data import HTTPResponse
from evo.notebooks import ServiceManagerWidget

cache_location = "./notebook-data"
download_path = cache_location + "/downloads"
input_path = f"{cache_location}/input"

if not os.path.exists(download_path):
    try:
        os.mkdir(download_path)
    except Exception:
        raise RuntimeError(f"Error: Failed to create {download_path}.")

# Evo app credentials
client_id = "<empty>"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

## Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

service_client = BlockModelAPIClient(environment, connector)

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

# # Create a BlockModelSelector instance. This will be used to select block models from the service.
# # The source code for this class is in helpers/evo_common.py
bm_selector = evo_common.BlockModelSelector(environment, connector)

## Demo 1: Create a regular block model

In [ ]:
from evo.blockmodels.data import RegularGridDefinition
from evo.blockmodels.endpoints.models import RotationAxis

block_grid = RegularGridDefinition(
    model_origin=[1478500, 5174500, 100],
    rotations=[(RotationAxis.x, 20)],
    n_blocks=[48, 68, 40],
    block_size=[25, 25, 25],
)

block_model, version = await service_client.create_block_model(
    name=f"My regular block model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    description="This is a description of my regular block model.",
    grid_definition=block_grid,
    object_path="/Jupyter/BlockModels/",
    coordinate_reference_system="EPSG:4326",
    size_unit_id="m",
)

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
# api_response = await connector.call_api(
#     method="GET",
#     resource_path=job_url,
#     response_types_map={
#         "200": HTTPResponse,
#     },
# )

# response = api_response.data.decode('utf-8')
# response_json = json.loads(response)
# print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style='lightbulb')))
bm_uuid = None

## Demo 2: Add columns to a block model

### Tell the API what changes we are making. In this example we are adding 2 new columns.

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {
    "columns": {
        "new": [
            {"title": "Geology", "data_type": "Utf8"},
            {"title": "AG_gpt", "data_type": "Float64"},
        ],
        "delete": [],
        "update": [],
        "rename": [],
    },
    "comment": "Comment updated during 'add' request.",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to add columns to the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Upload the block model data to the provided `upload_url`

In [ ]:
# Compose the headers
headers = {"Content-Type": "application/binary", "x-ms-blob-type": "BlockBlob"}

# Make a PUT request - include the binary data and headers
with open("sample/data.parquet", "rb") as data_stream:
    response = requests.put(url=upload_url, data=data_stream, headers=headers)

if response.status_code != HTTPStatus.CREATED:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response.json()}")

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 3: Display block models in your workspace

In [ ]:
await bm_selector.display_blockmodels()

## Demo 4: Download the latest version of a block model

In [ ]:
# Get the selected block model from the dropdown in the previous cell
selected_blockmodel = bm_selector.get_selected_item()
bm_uuid = selected_blockmodel["bm_uuid"]

# Part 1 - Send the initial request
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {"org_id": org_id, "workspace_id": workspace_id, "bm_uuid": bm_uuid}

body = {"columns": ["*"], "geometry_columns": "coordinates"}

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "200": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to download the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

### Check the job status until it is complete

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

if response_json["job_status"] == "COMPLETE":
    download_url = response_json["payload"]["download_url"]

### Download the data

In [ ]:
download_response = requests.get(download_url)
file_path = f"{download_path}/query_results.parquet"
open(file_path, "wb").write(download_response.content)

df = pd.read_parquet(file_path)
df.head()

## Demo 5: Download a block model using a bounding box

In [ ]:
# Get the selected block model from the dropdown in the previous cell
selected_blockmodel = bm_selector.get_selected_item()
bm_uuid = selected_blockmodel["bm_uuid"]

# Part 1 - Send the initial request
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {"org_id": org_id, "workspace_id": workspace_id, "bm_uuid": bm_uuid}

body = {
    "bbox": {
        "i_minmax": {"min": 3, "max": 6},
        "j_minmax": {"min": 2, "max": 8},
        "k_minmax": {"min": 1, "max": 3},
    },
    "columns": ["AG_gpt"],
}

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "200": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to download the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

### Check the job status until it is complete

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

if response_json["job_status"] == "COMPLETE":
    download_url = response_json["payload"]["download_url"]

### Download the data

In [ ]:
download_response = requests.get(download_url)
file_path = f"{download_path}/query_results.parquet"
open(file_path, "wb").write(download_response.content)

df = pd.read_parquet(file_path)
df.head()

## Demo 6: Display the list of available units

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/units"

path_params = {
    "org_id": org_id,
}

api_response = await connector.call_api(
    method="GET",
    resource_path=resource_path,
    path_params=path_params,
    response_types_map={
        "200": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to list units. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 7: Update units on existing columns

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {"org_id": org_id, "workspace_id": workspace_id, "bm_uuid": bm_uuid}

body = {
    "columns": {
        "new": [],
        "delete": [],
        "update": [],
        "rename": [],
        "update_metadata": [{"title": "AG_gpt", "values": {"unit_id": "%[mass]"}}],
    },
    "comment": "Updated AG_gpt unit.",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to update units of the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 8: Add new random columns

NOTE: You must run **Demo 5** before running this demo.

### Create the random data

In [ ]:
df["Random numbers"] = np.random.uniform(0, 100, size=len(df))
df["Random strings"] = np.random.choice(["Alpha", "Bravo", "Charlie", "Delta"], size=len(df))
temp_df = df[["i", "j", "k", "Random numbers", "Random strings"]]
temp_file_path = f"{download_path}/temp_query_results.parquet"
temp_df.to_parquet(temp_file_path, index=False)

### Perform the patch request

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {
    "columns": {
        "new": [
            {"title": "Random numbers", "data_type": "Float64"},
            {"title": "Random strings", "data_type": "Utf8"},
        ],
        "delete": [],
        "update": [],
        "rename": [],
    },
    "comment": "Added new columns.",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to add new columns to the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Upload the block model data to the provided `upload_url`

In [ ]:
headers = {"Content-Type": "application/binary", "x-ms-blob-type": "BlockBlob"}
with open(temp_file_path, "rb") as data_stream:
    response = requests.put(url=upload_url, data=data_stream, headers=headers)

if response.status_code != HTTPStatus.CREATED:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response.json()}")

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 9: Update a column and publish a new version of the block model

In [ ]:
column_to_update = "AG_gpt"

df[column_to_update] = df[column_to_update] + 1
df.head()
df.to_parquet(file_path, index=False)

resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {
    "columns": {"new": [], "delete": [], "update": [column_to_update], "rename": []},
    "comment": f"Updated the {column_to_update} column",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to update the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Upload the block model data to the provided `upload_url`

In [ ]:
# Compose the headers
headers = {"Content-Type": "application/binary", "x-ms-blob-type": "BlockBlob"}

# Make a PUT request - include the binary data and headers
with open(file_path, "rb") as data_stream:
    response = requests.put(url=upload_url, data=data_stream, headers=headers)

if response.status_code != HTTPStatus.CREATED:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response.json()}")

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 10: Rename columns

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {
    "columns": {
        "new": [],
        "delete": [],
        "update": [],
        "rename": [
            {"title": "AG_gpt", "new_title": "AG_gpt_renamed"},
            {"title": "Geology", "new_title": "Geology_renamed"},
        ],
    },
    "comment": "Updated column names.",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to rename columns in the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 11: Delete a column

In [ ]:
# Delete a column from a block model
column_to_delete = "AG_gpt_renamed"

resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {
    "columns": {"new": [], "delete": [column_to_delete], "update": [], "rename": []},
    "comment": "Deleted a column.",
}

api_response = await connector.call_api(
    method="PATCH",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "202": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.ACCEPTED:
    raise RuntimeError(f"Error: Failed to delete a column from the block model. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Copy the upload URL for later use
upload_url = response_json["upload_url"]

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Tell BlockSync that the patch is complete

In [ ]:
resource_path = job_url + "/uploaded"

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    response_types_map={
        "201": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 12: List all versions of a block model

In [ ]:
resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/versions"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

api_response = await connector.call_api(
    method="GET",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    response_types_map={
        "200": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to list block model versions. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
version_output = json.loads(response)

# Print the response in a highlighted format
print(highlight(json.dumps(version_output, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

## Demo 13: Download a specific block model version

In [ ]:
version_id = 2

# Parse the version_output from the previous cell to find the version_uuid for the specified version_id
filtered_version = next((v for v in version_output["results"] if v["version_id"] == version_id), None)
if not filtered_version:
    raise ValueError(f"Version {version_id} not found in the version_output.")

version_uuid = filtered_version["version_uuid"]

resource_path = "/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{bm_uuid}/blocks"

path_params = {
    "org_id": org_id,
    "workspace_id": workspace_id,
    "bm_uuid": bm_uuid,
}

body = {"columns": ["*"], "geometry_columns": "coordinates", "version_uuid": version_uuid}

api_response = await connector.call_api(
    method="POST",
    resource_path=resource_path,
    path_params=path_params,
    header_params={"Content-Type": "application/json"},
    body=body,
    response_types_map={
        "200": HTTPResponse,
    },
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to download block model version. Status: {status}")

# Parse the response data
response = api_response.data.decode("utf-8")
response_json = json.loads(response)

# Copy the job URL and remove the host portion to be compatible with the Evo connector object
job_url = "/" + "/".join(response_json["job_url"].split("/")[3:])

# Print the response in a highlighted format
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

### Keep checking the `job_url` until the `job_status` is `COMPLETE`

In [ ]:
api_response = await connector.call_api(
    method="GET",
    resource_path=job_url,
    response_types_map={
        "200": HTTPResponse,
    },
)

response = api_response.data.decode("utf-8")
response_json = json.loads(response)
print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

if response_json["job_status"] == "COMPLETE":
    download_url = response_json["payload"]["download_url"]

### Download the data

In [ ]:
download_response = requests.get(download_url)
file_path = f"{download_path}/query_results.parquet"
open(file_path, "wb").write(download_response.content)

df = pd.read_parquet(file_path)
df.head()